<hr style="border-width:2px;border-color:#75DFC1">
<center><h1>Projet Data Science</h1></center>
<center><h2>Création de jeu de données - Visiteurs</h2></center>
<hr style="border-width:2px;border-color:#75DFC1">

<blockquote>
Ici nous allons créer un jeu de données qui nous permettra d'étudier le comportement des visiteurs en fonction de leurs nombres d'interactions, etc, ...

</blockquote>

* charger le jeu de données <code>events_index.csv</code> dans un <code>pd.DataFrame</code> nommé <code>df</code>
* charger le jeu de données <code>events_transactions_clean.csv</code> dans un <code>pd.DataFrame</code> nommé <code>df_transactions_clean</code>

In [19]:
# Insérer le code ici


In [31]:
# Insérer le code ici

import pandas as pd 
import numpy as np

df = pd.read_csv('events_index.csv',
                # nrows=1000
                )
df_transactions_clean = pd.read_csv('events_transactions_clean.csv', 
                                   # nrows=1000
                                   )

/home/paul/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<blockquote>
    Maintenons, nous allons faire pivoter la table <code>df</code>. En effet, nous voulons une colonne <code>view</code> qui contient l'identifiant du produit vu, une colonne <code>addtocart</code> et une colonne <code>transaction</code> qui fonctionnent sur le même principe. Pour celà, nous pouvons utiliser la fonction <code>pd.pivot</code>.
    </blockquote>

* utiliser la fonction <code>pd.melt</code> pour réaliser l'opération demandée et stocker le résultat dans un <code>pd.DataFrame</code> nommé <code>df_pivot</code>. On passera la colonne <code>event_id</code> en index.
* joindre <code>df</code> et <code>df_pivot</code> en écrasant <code>df</code> grâce à la colonne <code>event_id</code>

In [32]:
# Insérer le code ici


In [33]:
# Insérer le code ici

df_pivot = pd.pivot(df, index='event_id', columns='event', values='itemid')

df = pd.merge(left=df, right=df_pivot, right_index=True, left_on='event_id', how='left')

df.head()

,timestamp,visitorid,event,itemid,transactionid,event_id,addtocart,transaction,view
0,1433221332117,257597,view,355908,NaN,0,NaN,NaN,355908.0
1,1433224214164,992329,view,248676,NaN,1,NaN,NaN,248676.0
2,1433221999827,111016,view,318965,NaN,2,NaN,NaN,318965.0
3,1433221955914,483717,view,253185,NaN,3,NaN,NaN,253185.0
4,1433221337106,951259,view,367447,NaN,4,NaN,NaN,367447.0


<blockquote>
    Maintenant nous allons aggréger les données concernant les transactions nettoyées par utilisateur.
</blockquote>

* aggréger <code>df_transactions_clean</code> par <code>visitor</code> en prenant le nombre de <code>transactionid</code> par utilisateur.
* attribuer ce résultat en écrasant <code>df_transactions_clean</code>

In [22]:
# Insérer le code ici


In [48]:
# Insérer le code ici

df_transactions_clean = df_transactions_clean.groupby(['visitorid']).agg({'transactionid': 'nunique'})
df_transactions_clean.head()

,transactionid
visitorid,
0,0
1,0
2,0
3,0
4,0


<blockquote>
    Aggrégeons à présent <code>df</code> par <code>visitorid</code>. 
    On va prendre les valeurs fonctions suivantes: 
        <ul>
        <li><code>itemid</code>: nombre de valeurs uniques non vides (nombre de produits avec lesquels le visiteur a interagi)</li>
        <li><code>view</code>: nombre de valeurs uniques non vides (nombres de produits différents vus) et nombre de valeurs non nulles (nombre de vues de produits): pour cette seconde option on pourra prendre la chaîne de caractères <code>size</code></li>
        <li><code>addtocart</code>: nombre de valeurs uniques non vides (nombres de produits différents ajoutés au panier) et nombre de valeurs non nulles (nombre d'ajouts au panier de produits)</li>
        <li><code>transaction</code>: nombre de valeurs uniques non vides (nombres de produits différents achetés) et nombre de valeurs non nulles (nombre d'achats de produits)</li>
        <li><code>timestamp</code>: valeur minimale et valeur maximale (date de la premiere interaction et date de la derniere interaction)    
</blockquote>

* construire un dictionnaire nommé <code>aggregation</code> qui permet de construire ces données.
* réaliser cette aggrégation en utilisant la méthode <code>groupby</code>
* renommer les colonnes avec les noms respectifs <code>['nb_products', 'nb_unique_views', 'nb_views', 'nb_unique_addtocarts', 'nb_addtocarts', 'nb_unique_buys', 'nb_buys', 'timestamp_min', 'timestamp_max']</code>

In [22]:
# Insérer le code ici


In [ ]:
# Insérer le code ici

aggregation = {
    'itemid': 'nunique',
    'view': ['nunique', 'size'],
    'addtocart': ['nunique', 'size'],
    'transaction': ['nunique', 'size'],
    'timestamp': ['min', 'max']
}

df = df.groupby(['visitorid']).agg(aggregation)

df.columns = ['nb_products', 'nb_unique_views', 'nb_views', 'nb_unique_addtocarts', 'nb_addtocarts', 'nb_unique_buys', 'nb_buys', 'timestamp_min', 'timestamp_max']

df.head()

<blockquote>
    Enfin, nous pouvons terminer en reliant les deux jeux de données.
</blockquote>

* renommer la colonne <code>transactionid</code> du jeu de données <code>df_transactions_clean</code> en <code>nb_unique_transactions</code>
* joindre les jeux de données grâce à la colonne <code>visitorid</code>
* sauvegarder les données dans un fichier <code>visitors.csv</code> grâce à la méthode <code>to_csv</code>

In [22]:
# Insérer le code ici


In [50]:
# Insérer le code ici

df_transactions_clean.columns = ['nb_unique_transactions']

df = pd.merge(left=df, right=df_transactions_clean, how='left', right_index=True, left_index=True)

df.to_csv('visitors.csv')

df.head()

,nb_products,nb_unique_views,nb_views,nb_unique_addtocarts,nb_addtocarts,nb_unique_buys,nb_buys,timestamp_min,timestamp_max,nb_unique_transactions
visitorid,,,,,,,,,,
0,3,3,3,1,3,1,3,1442004589439,1442004917175,0
1,1,1,1,1,1,1,1,1439487966444,1439487966444,0
2,4,4,8,1,8,1,8,1438969904567,1438971657845,0
3,1,1,1,1,1,1,1,1438413035296,1438413035296,0
4,1,1,1,1,1,1,1,1442352267167,1442352267167,0
